# Mask R-CNN Demo

A quick intro to using the pre-trained model to detect and segment objects.

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project

# ROOT_DIR = os.path.abspath("D:\keras\Matterpot_Mask_RCNN_Balloon")
ROOT_DIR = os.path.abspath("C:/Users/ereme/work/personal/Crack-Detection-TF-1.x.x")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import Crack config
sys.path.append(os.path.join(ROOT_DIR, "samples/crack"))  # To find local version
import crack

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
CRACK_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_crack.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(CRACK_MODEL_PATH):
    utils.download_trained_weights(CRACK_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "Dataset1/val")

## Configurations

We'll be using a model trained on the MS-COCO dataset. The configurations of this model are in the ```CocoConfig``` class in ```coco.py```.

For inferencing, modify the configurations a bit to fit the task. To do so, sub-class the ```CocoConfig``` class and override the attributes you need to change.

In [ ]:
class InferenceConfig(crack.CrackConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Create Model and Load Trained Weights

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(CRACK_MODEL_PATH, by_name=True)

## Class Names

The model classifies objects and returns class IDs, which are integer value that identify each class. Some datasets assign integer values to their classes and some don't. For example, in the Crack dataset, the 'crack' class is 1.

To get the list of class names, you'd load the dataset and then use the ```class_names``` property like this.
```
# Load Crack dataset
dataset = crack.CrackDataset()
dataset.load_coco(CRACK_MODEL_PATH, "train")
dataset.prepare()

# Print class names
print(dataset.class_names)
```

In [ ]:
class_names = ['BG', 'crack']

## Run Object Detection

In [ ]:
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
print(file_names)

image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[0]))

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])